In [1]:
import pickle

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
%matplotlib inline
import pandas as pd

In [3]:
import os, glob
import seaborn as sns
sns.set(style="darkgrid")

In [4]:
from IPython.display import display, HTML, Image

In [5]:
from sklearn.model_selection import train_test_split


In [6]:
data =pd.read_csv('10_PC_02_LHS_500_54854_01_s1_G.csv')
data5k =pd.read_csv(r'F:\ML4ChemcialReactions\chemical5K\10_PC_02_LHS_5000_54854_01_s1_G.csv')

In [7]:
### Remove the space from the column names
data.columns =[col.strip() for col in data.columns]
data5k.columns =[col.strip() for col in data5k.columns]


## Calculate the ratio of CaO to SiO2

## Put all togehter

In [8]:

data['ratio'] = data['b(CaO)']/data['b(SiO2)']
data5k['ratio'] = data5k['b(CaO)']/data5k['b(SiO2)']

In [9]:
#group1 = data[data['ratio']>=1.60]
#group2 = data[(data['ratio']<1.60) & (data['ratio']>0.67)]
#group3 = data[data['ratio']<=0.67]
group5k_1 = data5k[(data5k['nPortlandite']>=0.001)& (data5k['nAmor-Sl']<=0.001)]  # with Portlandite, no Amor-S1
group5k_2 = data5k[(data5k['nPortlandite']<=0.001)& (data5k['nAmor-Sl']<=0.001)]   # no Portlandite, no Amor-S1
group5k_3 = data5k[(data5k['nPortlandite']<=0.001)& (data5k['nAmor-Sl']>=0.001)]    # no Portlandite, with Amor-S1

In [10]:
print(group5k_1['ratio'].max(),group5k_1['ratio'].min())  # with Portlandite, no Amor-S1
group1_low=group5k_1['ratio'].min()

8.540749112357153 1.6332348368965248


In [11]:
print(group5k_2['ratio'].max(),group5k_2['ratio'].min())   # no Portlandite, no Amor-S1

group2_upper=group5k_2['ratio'].max()
group2_low =group5k_2['ratio'].min()   # no Portlandite, no Amor-S1


1.6347578830913554 0.6733330997218754


In [12]:
print(group5k_3['ratio'].max(),group5k_3['ratio'].min()) # no Portlandite, with Amor-S1
group3_upper= group5k_3['ratio'].max()

0.6722489930692754 0.15561277244409333


In [13]:
## Devided 3 groups which are based on the Data distribution, particularly pH distribution
# ratio of 1.65 is based on 500-dataset. it seems that 1.60 is better based on 5K-dataset.
#This suggests that more data near to the boundary, more accurate the line can be determined
group1 = data[data['ratio']>=group1_low]  # with Portlandite, no Amor-S1
group2 = data[(data['ratio']<=group2_upper) & (data['ratio']>=group2_low)]  # no Portlandite, no Amor-S1
group3 = data[data['ratio']<=group3_upper]     # no Portlandite, with Amor-S1
group5k1 = data5k[data5k['ratio']>=group1_low]  # with Portlandite, no Amor-S1
group5k2 = data5k[(group2_low<=data5k['ratio']) & (data5k['ratio']<=group2_upper)]  # no Portlandite, no Amor-S1
group5k3 = data5k[data5k['ratio']<=group3_upper]    # no Portlandite, with Amor-S1

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
### Output the trainning dataset for dataset5K
group1_XY_train,group1_XY_test = train_test_split(group5k1.values,test_size=0.2, train_size=0.4,shuffle=True)
group2_XY_train,group2_XY_test = train_test_split(group5k2.values,test_size=0.2, train_size=0.4,shuffle=True)
group3_XY_train,group3_XY_test = train_test_split(group5k3.values,test_size=0.2, train_size=0.4,shuffle=True)
test_XY=np.concatenate((group1_XY_test,group2_XY_test,group3_XY_test))
### Save the data to local drive
testDF = pd.DataFrame(test_XY,columns=data.columns)
group1DF_train = pd.DataFrame(group1_XY_train,columns=data.columns)
group2DF_train = pd.DataFrame(group2_XY_train,columns=data.columns)
group3DF_train = pd.DataFrame(group3_XY_train,columns=data.columns)


## exclude the ratio column
testDF.loc[:,testDF.columns!='ratio'].to_csv('dataset5K_test3k.csv',index=False)
group1DF_train.loc[:,group1DF_train.columns!='ratio'].to_csv('Group5K1_trainning_dataset2k.csv',index=False)
group2DF_train.loc[:,group2DF_train.columns!='ratio'].to_csv('Group5K2_trainning_dataset2k.csv',index=False)
group3DF_train.loc[:,group3DF_train.columns!='ratio'].to_csv('Group5K3_trainning_dataset2k.csv',index=False)




In [ ]:
### This dataset will be saved for testing the models


In [17]:
group1_XY_train = group1.values
group2_XY_train = group2.values
group3_XY_train = group3.values

#test_XY=np.concatenate((group1_XY_test,group2_XY_test,group3_XY_test))
### Save the data to local drive
#testDF = pd.DataFrame(test_XY,columns=data.columns)
group1DF_train = pd.DataFrame(group1_XY_train,columns=data.columns)
group2DF_train = pd.DataFrame(group2_XY_train,columns=data.columns)
group3DF_train = pd.DataFrame(group3_XY_train,columns=data.columns)
## exclude the ratio column
#testDF.loc[:,testDF.columns!='ratio'].to_csv('dataset5H_test.csv',index=False)
group1DF_train.loc[:,group1DF_train.columns!='ratio'].to_csv('Group5H1_trainning_dataset_all.csv',index=False)
group2DF_train.loc[:,group2DF_train.columns!='ratio'].to_csv('Group5H2_trainning_dataset_all.csv',index=False)
group3DF_train.loc[:,group3DF_train.columns!='ratio'].to_csv('Group5H3_trainning_dataset_all.csv',index=False)


In [ ]:
### Visualizing the three groups 
fig, ax =plt.subplots(figsize=(12,8))
x1= np.linspace(0.0,0.9,30);
y1 = 0+group3_upper*x1
x2= np.linspace(0.0,0.9,30);
y2 = 0+group1_low*x2
#ax.plot(data5k['b(SiO2)'].values,data5k['b(CaO)'].values,'b.',markerfacecolor = 'tab:olive',markeredgecolor ='darkblue',markeredgewidth=0.5,label='data5k')
ax.plot(group5k1['b(SiO2)'].values,group5k1['b(CaO)'].values,'bo',markerfacecolor = 'tab:olive',markeredgecolor ='darkblue',markeredgewidth=0.5,label='group5k1')
ax.plot(group5k2['b(SiO2)'].values,group5k2['b(CaO)'].values,'bo',markerfacecolor = 'c',markeredgecolor ='b',markeredgewidth=0.5,label='group5k2')
ax.plot(group5k3['b(SiO2)'].values,group5k3['b(CaO)'].values,'g^',markerfacecolor = 'm',markeredgecolor ='m',markeredgewidth=1,label='group5k3')

ax.plot(group1['b(SiO2)'].values,group1['b(CaO)'].values,'bo',markerfacecolor = 'tab:olive',markeredgecolor ='darkblue',markeredgewidth=0.5,label='group5H1')
ax.plot(group2['b(SiO2)'].values,group2['b(CaO)'].values,'bo',markerfacecolor = 'c',markeredgecolor ='b',markeredgewidth=0.5,label='group5H2')
ax.plot(group3['b(SiO2)'].values,group3['b(CaO)'].values,'g^',markerfacecolor = 'm',markeredgecolor ='m',markeredgewidth=1,label='group5H3')
ax.plot(x1,y1,'r-',lw=2,label ='Line 1:CaO/SiO2 = '+str(group3_upper)[:5])
ax.plot(x2,y2,'y-',lw=3,label ='Line 2:CaO/SiO2 = '+str(group1_low)[:5])
ax.legend (loc='best',ncol=5)           
ax.set_xlim(0,0.9)
ax.set_ylim(0,2.2)
ax.text(0.1, 1.25, 'no Amor-Si, with Portlandite',fontsize = 20,color = 'brown')
ax.text(0.5, 0.62, 'no Amor-Si or Portlandite',fontsize = 20,color = 'k')
ax.text(0.35, 0.02, 'with Amor-Si no Portlandite',fontsize = 20,color = 'k')


ax.set_xlabel('b(SiO2)')
ax.set_ylabel('b(CaO)')

